<a href="https://colab.research.google.com/github/nanthony007/gfdx-analysis/blob/master/gfdx_redcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Hello there this is my GFDx notebook!

1.   FF Opportunity



---




In [55]:
!pip install PyCap

from redcap import Project
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # progress bar


URL = 'https://redcap.emory.edu/api/'
API_KEY = '5E21E0FE11A1F430927A1186D98F3E8F'
project = Project(URL, API_KEY)

In [56]:
df = project.export_records(format='df')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18,24,27,29,30,165,171,179,187,192,197,198,199,250,251,252,258,259,260,279,287,293,301,307,308,309,315,316,317,323,324,325,331,332,333,339,340,341,362,363,364,366,367,368) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
df.head()

redcap_repeat_instrument  ...  article_scientific
country_code redcap_event_name                           ...                    
1.0          all_foods_arm_1                        NaN  ...                 NaN
             maize_flour_arm_1                      NaN  ...                 NaN
             oil_arm_1                              NaN  ...                 NaN
             rice_arm_1                             NaN  ...                 NaN
             salt_arm_1                             NaN  ...                 NaN

[5 rows x 376 columns]

In [58]:
df.shape

(107253, 376)

In [59]:
df.reset_index(inplace=True)
df.head()

,country_code,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,country_name,country_name_short,country_territory,un_region,ffi_region,unicef_region,wb_income_status,wb_income_status_year,wb_income_status_source,population,population_year,population_source,percent_urban,percent_urban_year,percent_urban_source,anemia_ch_biomarker,anemia_ch_prevalence,anemia_ch_age_range,anemia_ch_year,anemia_ch_comment,anemia_ch_source,anemia_nonpreg_biomarker,anemia_nonpreg_prevalence,anemia_nonpreg_age_range,anemia_nonpreg_year,anemia_nonpreg_comment,anemia_nonpreg_source,anemia_men_biomarker,anemia_men_prevalence,anemia_men_age_range,anemia_men_year,anemia_men_comment,anemia_men_source,irondef_ch_biomarker,irondef_ch_prevalence,irondef_ch_age_range,...,coverage_ffv_hh_year,coverage_ffv_hh_source,cov_ffv_hh_source_english,coverage_ffv_hh_comment,coverage_ffv_hh_file_1,coverage_ffv_hh_file_2,coverage_ffv_hh_quant,coverage_ffv_hh_quant_data,coverage_ffv_hh_quant_year,cov_ffv_hh_quant_source,cov_ffv_hh_quant_source_e,cov_ffv_hh_quant_comment,cov_ffv_hh_quant_file_1,cov_ffv_hh_quant_file_2,intake_category_grains,intake_category_o,intake_category_s,coverage_fv_category,reach_potential,reach_potential_supp,operational_ease_category,operational_ease_supp,overall_recommendation,overall_rec_supp,article_author,article_title,article_journal,article_year,article_link,article_citation,article_version,article_external,article_internal_steward,article_internal_org,article_cited,article_data,article_revisualized,article_presentation,article_report,article_scientific
0,1.0,all_foods_arm_1,NaN,NaN,Armenia,Armenia,1.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN,World Health Organization. The global prevalen...,NaN,26.0,NaN,NaN,NaN,World Health Organization. The global prevalen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,maize_flour_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,oil_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,rice_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,salt_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import os
from redcap import Project
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # progress bar
from dotenv import load_dotenv
load_dotenv()



project = Project(
    os.getenv('URL'), 
    os.getenv('API_KEY')
)

In [70]:
df.shape

array([nan])

In [60]:
# Create list of unique food vehicles
food_list = ['maize_flour_arm_1', 'wheat_flour_arm_1', 'rice_arm_1', 'salt_arm_1', 'oil_arm_1']

In [61]:
# Filtered for data in the food_list
filt_df = df[df.redcap_event_name.isin(food_list)]
countries = filt_df.country_code.unique()  # Unique country codes from the filtered data
foods = filt_df.redcap_event_name.unique() # Unique food vehicles from filtered data

In [62]:
foods.sort() == food_list.sort()  # Should be true

True

In [63]:
targets = [
           {'instrument': 'intake', 'col': ['food_intake', 'import_pc']}, 
           {'instrument': 'coverage_fv', 'col': 'coverage_fv'},
           {'instrument': 'industrially_processed', 'col': 'industrially_processed_pc'}, 
           {'instrument': 'coverage_ipfv', 'col': 'coverage_ipfv' },
           ]

In [69]:
new_df = pd.DataFrame()
for country in tqdm(countries):
    for food in foods:
        for target in targets:
            sliced = filt_df[
                             (filt_df.country_code == country) & 
                             (filt_df.redcap_event_name == food) & 
                             (filt_df.redcap_repeat_instrument == target['instrument'])
                             ]
            if target['instrument'] == 'intake':
                single = sliced[sliced.redcap_repeat_instance == sliced.redcap_repeat_instance.max()][['country_code', 'redcap_event_name', 'redcap_repeat_instrument', 'food_intake', 'import_pc']]
                if len(single) == 0:
                    single_revised = pd.DataFrame(
                        [[country, food, target['instrument'], np.NaN, np.NaN]], 
                        columns=['country_code', 'redcap_event_name', 'redcap_repeat_instrument', 'food_intake', 'import_pc']
                    )
                    new_df = new_df.append(single_revised, ignore_index=True)
                else:
                    new_df = new_df.append(single, ignore_index=True)
            else:
                single = sliced[sliced.redcap_repeat_instance == sliced.redcap_repeat_instance.max()][['country_code', 'redcap_event_name', 'redcap_repeat_instrument', target['col']]]
                if len(single) == 0:
                    single_revised =  pd.DataFrame(
                        [[country, food, target['instrument'], np.NaN]], 
                        columns=['country_code', 'redcap_event_name', 'redcap_repeat_instrument', target['col']]
                    )
                    new_df = new_df.append(single_revised, ignore_index=True)
                else:
                    new_df = new_df.append(single, ignore_index=True)


In [67]:
# old stuff

df = pd.read_csv('/content/gfdx-stuff.csv')
df.head()

,country_code,redcap_event_name,redcap_repeat_instrument,food_intake,import_pc,coverage_fv,industrially_processed_pc,coverage_ipfv
0,1.0,maize_flour_arm_1,intake,20.49,NaN,NaN,NaN,NaN
1,1.0,maize_flour_arm_1,coverage_fv,NaN,NaN,NaN,NaN,NaN
2,1.0,maize_flour_arm_1,industrially_processed,NaN,NaN,NaN,100.0,NaN
3,1.0,maize_flour_arm_1,coverage_ipfv,NaN,NaN,NaN,NaN,NaN
4,1.0,oil_arm_1,intake,25.32,NaN,NaN,NaN,NaN
5,1.0,oil_arm_1,coverage_fv,NaN,NaN,NaN,NaN,NaN
6,1.0,oil_arm_1,industrially_processed,NaN,NaN,NaN,NaN,NaN
7,1.0,oil_arm_1,coverage_ipfv,NaN,NaN,NaN,NaN,NaN
8,1.0,rice_arm_1,intake,9.01,NaN,NaN,NaN,NaN
9,1.0,rice_arm_1,coverage_fv,NaN,NaN,NaN,NaN,NaN


In [68]:
new_df.import_pc.unique()

array([nan])

In [49]:
grains = ['maize_flour_arm_1', 'wheat_flour_arm_1', 'rice_arm_1']
oil = 'oil_arm_1'
salt = 'salt_arm_1'

In [50]:
def identify_availability(row):
    return_val = ''
    if row.redcap_event_name in grains:
        if row.food_intake < 75:
            return_val = 'Low'
        elif row.food_intake < 150:
            return_val = 'Moderate'
        elif row.food_intake < 300:
            return_val = 'Moderately High'
        elif row.food_intake >= 300:
            return_val = 'High'
        else:
            return_val = 'Unknown'
    elif row.redcap_event_name == oil:
        if row.food_intake < 20:
            return_val = 'Low'
        elif row.food_intake < 40:
            return_val = 'Moderate'
        elif row.food_intake < 60:
            return_val = 'Moderately High'
        elif row.food_intake >= 60:
            return_val = 'High'
        else:
            return_val = 'Unknown' 
    elif row.redcap_event_name == salt:
        return_val = 'High'
    else:
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val


def identify_coverage(row):
    return_val = ''
    if row.redcap_event_name in grains:
        if row.coverage_fv < 35:
            return_val = 'Low'
        elif row.coverage_fv < 75:
            return_val = 'Moderate'
        elif row.coverage_fv >= 75:
            return_val = 'High'
        else:
            return_val = 'Unknown'
    elif row.redcap_event_name in [oil, salt]:
        if row.coverage_fv < 35:
            return_val = 'Low'
        elif row.coverage_fv < 75:
            return_val = 'Moderate'
        else:
            return_val = 'High'
    else:
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val

In [71]:
new_df['food_avail'] = new_df.apply(lambda row: identify_availability(row), axis=1)
new_df['pop_coverage'] = new_df.apply(lambda row: identify_coverage(row), axis=1)
new_df.head()

,country_code,redcap_event_name,redcap_repeat_instrument,food_intake,import_pc,coverage_fv,industrially_processed_pc,coverage_ipfv,food_avail,pop_coverage
0,1.0,maize_flour_arm_1,intake,20.49,NaN,NaN,NaN,NaN,Low,Unknown
1,1.0,maize_flour_arm_1,coverage_fv,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown
2,1.0,maize_flour_arm_1,industrially_processed,NaN,NaN,NaN,100.0,NaN,Unknown,Unknown
3,1.0,maize_flour_arm_1,coverage_ipfv,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown
4,1.0,oil_arm_1,intake,25.32,NaN,NaN,NaN,NaN,Moderate,High


In [52]:
def assess_reach(x, y):
    return_val = ''
    if x == 'Low' and y == 'Low':
        return_val = 'Low'
    elif x == 'Low' and y == 'Moderate':
        return_val = 'Moderate'
    elif x == 'Low' and y == 'High':
        return_val = 'Moderate'
    elif x == 'Moderate' and y == 'Low':
        return_val = 'Moderate'
    elif x == 'Moderate' and y == 'Moderate':
        return_val = 'Moderate'
    elif x == 'Moderate' and y == 'High':
        return_val = 'High'
    elif x == 'Moderately High' and y == 'Low':
        return_val = 'Moderate'
    elif x == 'Moderately High' and y == 'Moderate':
        return_val = 'Moderate'
    elif x == 'Moderately High' and y == 'High':
        return_val = 'High'
    elif x == 'High' and y == 'Low':
        return_val = 'Moderate'
    elif x == 'High' and y == 'Moderate':
        return_val = 'High'
    elif x == 'High' and y == 'High':
        return_val = 'High'
    elif x == 'Unknown' and y == 'Unknown':
        return_val = 'Unknown'
    # recurse
    # elif x == 'Unknown' and y != 'Unknown':
    #     assess_reach(y, y)
    # # recurse
    # elif x != 'Unknown' and y == 'Unknown':
    #     assess_reach(x, x)
    # michi
    elif x == 'Unknown' and y == 'Low':
        return_val = 'Low'
    elif x == 'Unknown' and y == 'Moderate':
        return_val = 'Moderate'
    elif x == 'Unknown' and y == 'High':
        return_val = 'High'
    elif x == 'Low' and y == 'Unknown':
        return_val = 'Low'
    elif x == 'Moderate' and y == 'Unknown':
        return_val = 'Moderate'
    elif x == 'Moderately High' and y == 'Unknown':
        return_val = 'Moderate'
    elif x == 'High' and y == 'Unknown':
        return_val = 'High'
    return return_val


In [84]:
def assess_industry(df):
    industrial_var = df[df.redcap_repeat_instrument == 'industrially_processed'].industrially_processed_pc.iloc[0]
    if industrial_var < 35:
        return 'Significant'
    elif industrial_var < 75:
        return 'Some'
    elif industrial_var >= 75:
        return 'Minimal'
    elif industrial_var == np.NaN:
            coverage_var = df[df.redcap_repeat_instrument == 'coverage_ipfv'].coverage_ipfv.iloc[0]
            if coverage_var < 35:
                return 'Significant'
            elif coverage_var < 75:
                return 'Some'
            elif coverage_var >= 75:
                return 'Minimal'
            elif coverage_var == np.NaN:
                import_var = df[df.redcap_repeat_instrument == 'intake'].import_pc.iloc[0]
                if import_var < 35:
                    return 'Significant'
                elif import_var < 75:
                    return 'Some'
                elif import_var >= 75:
                    return 'Minimal'
    else:
        return 'Unknown'

In [85]:
new_df.head()

,country_code,redcap_event_name,redcap_repeat_instrument,food_intake,import_pc,coverage_fv,industrially_processed_pc,coverage_ipfv,food_avail,pop_coverage
0,1.0,maize_flour_arm_1,intake,20.49,NaN,NaN,NaN,NaN,Low,Unknown
1,1.0,maize_flour_arm_1,coverage_fv,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown
2,1.0,maize_flour_arm_1,industrially_processed,NaN,NaN,NaN,100.0,NaN,Unknown,Unknown
3,1.0,maize_flour_arm_1,coverage_ipfv,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown
4,1.0,oil_arm_1,intake,25.32,NaN,NaN,NaN,NaN,Moderate,High


In [86]:
new_countries = []
new_foods = []
food_availabilities = []
industry_processes = []
pop_coverages = []
reaches = []
for country in new_df.country_code.unique():
    for food in new_df.redcap_event_name.unique():
            temp = new_df[
                            (new_df.country_code == country) & 
                            (new_df.redcap_event_name == food)
                            ]

            food_avail = temp[temp.redcap_repeat_instrument == 'intake'].food_avail.iloc[0]
            pop_cover = temp[temp.redcap_repeat_instrument == 'coverage_fv'].pop_coverage.iloc[0]

            new_countries.append(country)
            new_foods.append(food)

            food_availabilities.append(food_avail)
            pop_coverages.append(pop_cover)

            industry_processes.append(assess_industry(temp))

            reaches.append(assess_reach(food_avail, pop_cover))

In [87]:
def assess_overall(row):
    if row.reaches == 'High' and row.industry_processes == 'Minimal':
        return 'Good'
    elif row.reaches == 'High' and row.industry_processes == 'Some':
        return 'Good WC'
    elif row.reaches == 'High' and row.industry_processes == 'Significant':
        return 'Moderate WC'
    elif row.reaches == 'High' and row.industry_processes == 'Unknown':
        return 'Moderate WC'
    elif row.reaches == 'Moderate' and row.industry_processes == 'Minimal':
        return 'Good'
    elif row.reaches == 'Moderate' and row.industry_processes == 'Some':
        return 'Good WC'
    elif row.reaches == 'Moderate' and row.industry_processes == 'Significant':
        return 'Moderate WC'
    elif row.reaches == 'Moderate' and row.industry_processes == 'Unknown':
        return 'Moderate WC'
    elif row.reaches == 'Low' and row.industry_processes == 'Minimal':
        return 'Moderate WC'
    elif row.reaches == 'Low' and row.industry_processes == 'Some':
        return 'Moderate WC'
    elif row.reaches == 'Low' and row.industry_processes == 'Significant':
        return 'Poor'
    elif row.reaches == 'Low' and row.industry_processes == 'Unknown':
        return 'Poor'
    elif row.reaches == 'Unknown' and row.industry_processes == 'Minimal':
        return 'Moderate'
    elif row.reaches == 'Unknown' and row.industry_processes == 'Some':
        return 'Moderate'
    elif row.reaches == 'Unknown' and row.industry_processes == 'Significant':
        return 'Poor'
    elif row.reaches == 'Unknown' and row.industry_processes == 'Unknown':
        return 'Unknown'


In [90]:
final = pd.DataFrame()
final['country_code'] = new_countries
final['foods'] = new_foods
final['food_avails'] = food_availabilities
final['industry_processes'] = industry_processes
final['pop_covers'] = pop_coverages
final['reaches'] = reaches
final['overall'] = final.apply(lambda row: assess_overall(row), axis=1)

final.head()

,country_code,foods,food_avails,pop_covers,reaches,industry_processes,overall
0,1.0,maize_flour_arm_1,Low,Unknown,Low,Minimal,Moderate WC
1,1.0,oil_arm_1,Moderate,High,High,Unknown,Moderate WC
2,1.0,rice_arm_1,Low,Unknown,Low,Minimal,Moderate WC
3,1.0,salt_arm_1,High,High,High,Unknown,Moderate WC
4,1.0,wheat_flour_arm_1,High,Unknown,High,Minimal,Good


In [ ]:
final

In [ ]:
final.to_csv('i did this.csv')